In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My\ Drive/PA2

In [ ]:
!pip install lime
!pip install pytorch_transformers==1.2.0

In [ ]:
pip install transformers

In [ ]:
pip install ktrain

In [ ]:
from lime.lime_text import LimeTextExplainer
label_names = [0, 1]
explainer = LimeTextExplainer(class_names=label_names)

In [ ]:
import ktrain 
from ktrain import text
import pandas as pd


#Il modello addestrato di Bert viene caricato qua
t=text.Transformer('./experiments/CAMERA_SPEC__BERT__20200904_105341', maxlen=128, classes=label_names)


# #train
train = pd.read_csv("data/camera_spec/train.tsv",sep="\t")


#ciclo train 
train_list = []
label_train_list=[]
for i in range(1,4999,1):
    x_train = train.left_spec_title[i] + " " + train.right_spec_title[i]
    train_list.append(x_train)
    y_train = train.label[i]
    label_train_list.append(y_train)

# #test
test = pd.read_csv("data/camera_spec/test.tsv",sep="\t")


#ciclo test
test_list = []
label_test_list=[]
for i in range(1,999,1):
    x_test = test.left_spec_title[i] + " " + test.right_spec_title[i]
    test_list.append(x_test)
    y_test = test.label[i]
    label_test_list.append(y_test)

trn = t.preprocess_train(train_list, label_train_list)
val = t.preprocess_test(test_list, label_test_list)
modello = t.get_classifier()
learner = ktrain.get_learner(modello, train_data=trn, val_data=val, batch_size=3)

In [ ]:
learner.fit_onecycle(5e-5, 4)

In [ ]:
#predizioni
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
#PYSPARK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz 
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My\ Drive/PA2/spark-3.0.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()



In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc=t)
predictor.save("/tmp/ultima")

In [ ]:
def explanation(record):
  new_predictor = ktrain.load_predictor("/tmp/ultima")
  new_predictor.predict(record)
  new_predictor.explain(record)
  new_predictor.save("/tmp/my_predictor")
  reloaded_predictor = ktrain.load_predictor("/tmp/my_predictor")
  reloaded_predictor.predict_proba(record)
  exp = explainer.explain_instance(record, reloaded_predictor.predict_proba, num_features=6)
  return exp.as_list()
  

In [ ]:
lines= sc.textFile("test.tsv").map(lambda line: line.split('\t'))
rdd = lines.map(lambda x:(x[3], x[0]+ " " +x[1]))
spieg = rdd.map(lambda l: (explanation(l[1])))


In [ ]:
result= spieg.flatMap(lambda x: x)

In [ ]:
result.take(10)

In [ ]:
sc.stop()
spark.stop()